In [1]:
'''
MIT License
Copyright (c) 2019 Fanjin Zeng
This work is licensed under the terms of the MIT license, see <https://opensource.org/licenses/MIT>.  
https://gist.github.com/Fnjn/58e5eaa27a3dc004c3526ea82a92de80
RRT = rapidly exploring random tree
'''

import numpy as np
from random import random
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
from collections import deque
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
import ipympl
%matplotlib widget

In [51]:
# 3D version in progress
def dijkstra(G):
    '''
    Dijkstra algorithm for finding shortest path from start position to end.
    '''
    srcIdx = G.vex2idx[G.startpos]
    dstIdx = G.vex2idx[G.endpos]

    # build dijkstra
    nodes = list(G.neighbors.keys())
    dist = {node: float('inf') for node in nodes}
    prev = {node: None for node in nodes}
    dist[srcIdx] = 0

    while nodes:
        curNode = min(nodes, key=lambda node: dist[node])
        nodes.remove(curNode)
        if dist[curNode] == float('inf'):
            break

        for neighbor, cost in G.neighbors[curNode]:
            newCost = dist[curNode] + cost
            if newCost < dist[neighbor]:
                dist[neighbor] = newCost
                prev[neighbor] = curNode

    # retrieve path
    path = deque()
    curNode = dstIdx
    while prev[curNode] is not None:
        path.appendleft(G.vertices[curNode])
        curNode = prev[curNode]
    path.appendleft(G.vertices[curNode])
    return list(path)

class Line():
    ''' Define line '''
    def __init__(self, p0, p1):
        self.p = np.array(p0)
        self.dirn = np.array(p1) - np.array(p0)
        self.dist = np.linalg.norm(self.dirn)
        self.dirn /= self.dist # normalize

    def path(self, t):
        return self.p + t * self.dirn


def Intersection(line, center, radius):
    ''' Check line-sphere (circle) intersection '''
    a = np.dot(line.dirn, line.dirn)
    b = 2 * np.dot(line.dirn, line.p - center)
    c = np.dot(line.p - center, line.p - center) - radius * radius

    discriminant = b * b - 4 * a * c
    if discriminant < 0:
        return False

    t1 = (-b + np.sqrt(discriminant)) / (2 * a);
    t2 = (-b - np.sqrt(discriminant)) / (2 * a);

    if (t1 < 0 and t2 < 0) or (t1 > line.dist and t2 > line.dist):
        return False

    return True


def distance(obj1, obj2):
    return np.linalg.norm(np.array(obj1) - np.array(obj2))


def isInObstacle(vex, obstacles, radius):
    for obs in obstacles:
        if distance(obs, vex) < radius:
            return True
    return False


def isThruObstacle(line, obstacles, radius):
    for obs in obstacles:
        if Intersection(line, obs, radius):
            return True
    return False


def nearest(G, vex, obstacles, radius):
    Nvex = None
    Nidx = None
    minDist = float("inf")

    for idx, v in enumerate(G.vertices):
        line = Line(v, vex)
        if isThruObstacle(line, obstacles, radius):
            continue

        dist = distance(v, vex)
        if dist < minDist:
            minDist = dist
            Nidx = idx
            Nvex = v

    return Nvex, Nidx


def newVertex(randvex, nearvex, stepSize):
    dirn = np.array(randvex) - np.array(nearvex)
    length = np.linalg.norm(dirn)
    dirn = (dirn / length) * min (stepSize, length)

    newvex = (nearvex[0]+dirn[0], nearvex[1]+dirn[1], nearvex[2] + dirn[2])
    return newvex


def window(startpos, endpos):
    ''' Define seach window - 2 times of start to end rectangle'''
    width = endpos[0] - startpos[0]
    height = endpos[1] - startpos[1]
    winx = startpos[0] - (width / 2.)
    winy = startpos[1] - (height / 2.)
    return winx, winy, width, height


def isInWindow(pos, winx, winy, width, height):
    ''' Restrict new vertex insides search window'''
    if winx < pos[0] < winx+width and \
        winy < pos[1] < winy+height:
        return True
    else:
        return False


class Graph:
    ''' Define graph '''
    def __init__(self, startpos, endpos):
        self.startpos = startpos
        self.endpos = endpos

        self.vertices = [startpos]
        self.edges = []
        self.success = False

        self.vex2idx = {startpos:0}
        self.neighbors = {0:[]}
        self.distances = {0:0.}

        self.sx = endpos[0] - startpos[0]
        self.sy = endpos[1] - startpos[1]
        self.sz = endpos[2] - startpos[2]

    def add_vex(self, pos):
        try:
            idx = self.vex2idx[pos]
        except:
            idx = len(self.vertices)
            self.vertices.append(pos)
            self.vex2idx[pos] = idx
            self.neighbors[idx] = []
        return idx

    def add_edge(self, idx1, idx2, cost):
        self.edges.append((idx1, idx2))
        self.neighbors[idx1].append((idx2, cost))
        self.neighbors[idx2].append((idx1, cost))


    def randomPosition(self):
        rx = random()
        ry = random()
        rz = random()

        posx = self.startpos[0] - (self.sx / 2.) + rx * self.sx * 2
        posy = self.startpos[1] - (self.sy / 2.) + ry * self.sy * 2
        posz = self.startpos[2] - (self.sz / 2.) + rz * self.sz * 2
        return posx, posy, posz


def RRT(startpos, endpos, obstacles, n_iter, radius, stepSize):
    ''' RRT algorithm '''
    G = Graph(startpos, endpos)

    for _ in range(n_iter):
        randvex = G.randomPosition()
        if isInObstacle(randvex, obstacles, radius):
            continue

        nearvex, nearidx = nearest(G, randvex, obstacles, radius)
        if nearvex is None:
            continue

        newvex = newVertex(randvex, nearvex, stepSize)

        newidx = G.add_vex(newvex)
        dist = distance(newvex, nearvex)
        G.add_edge(newidx, nearidx, dist)

        dist = distance(newvex, G.endpos)
        if dist < 2 * radius:
            endidx = G.add_vex(G.endpos)
            G.add_edge(newidx, endidx, dist)
            G.success = True
            #print('success')
            # break
    return G


def RRT_star(startpos, endpos, obstacles, n_iter, radius, stepSize):
    ''' RRT star algorithm '''
    G = Graph(startpos, endpos)

    for _ in range(n_iter):
        randvex = G.randomPosition()
        if isInObstacle(randvex, obstacles, radius):
            continue

        nearvex, nearidx = nearest(G, randvex, obstacles, radius)
        if nearvex is None:
            continue

        newvex = newVertex(randvex, nearvex, stepSize)

        newidx = G.add_vex(newvex)
        dist = distance(newvex, nearvex)
        G.add_edge(newidx, nearidx, dist)
        G.distances[newidx] = G.distances[nearidx] + dist

        # update nearby vertices distance (if shorter)
        for vex in G.vertices:
            if vex == newvex:
                continue

            dist = distance(vex, newvex)
            if dist > radius:
                continue

            line = Line(vex, newvex)
            if isThruObstacle(line, obstacles, radius):
                continue

            idx = G.vex2idx[vex]
            if G.distances[newidx] + dist < G.distances[idx]:
                G.add_edge(idx, newidx, dist)
                G.distances[idx] = G.distances[newidx] + dist

        dist = distance(newvex, G.endpos)
        if dist < 2 * radius:
            endidx = G.add_vex(G.endpos)
            G.add_edge(newidx, endidx, dist)
            try:
                G.distances[endidx] = min(G.distances[endidx], G.distances[newidx]+dist)
            except:
                G.distances[endidx] = G.distances[newidx]+dist

            G.success = True
            #print('success')
            # break
    return G


def plt_sphere(list_center, list_radius, ax):
    for c, r in zip(list_center, list_radius):
        # draw sphere
        u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
        x = r*np.cos(u)*np.sin(v)
        y = r*np.sin(u)*np.sin(v)
        z = r*np.cos(v)
        ax.plot_surface(c[0] - x, c[1] - y, c[2] - z, color="red")


def plot(G, obstacles, radius, path=None):
    '''
    Plot RRT, obstacles and shortest path. obstacles are 4d stuff
    '''
    plt.close()
    px = [x for x, y, z in G.vertices]
    py = [y for x, y, z in G.vertices]
    pz = [z for x, y, z in G.vertices]
    ax = plt.axes(projection='3d')
    #fig, ax = plt.subplots()

    #for obs in obstacles:
        #circle = plt.Circle(obs, radius, color='red')
        #ax.add_artist(circle)
        #ax.scatter3D(obs[0], obs[1], obs[2], c="red", s=radius)
    plt_sphere(obstacles, [radius]*len(obstacles), ax)

    ax.scatter3D(px, py, pz, c='cyan')
    ax.scatter3D(G.startpos[0], G.startpos[1], G.startpos[2], c='black')
    ax.scatter3D(G.endpos[0], G.endpos[1], G.endpos[2], c='black')

    lines = [(G.vertices[edge[0]], G.vertices[edge[1]]) for edge in G.edges]
    print(G.edges)
    #lc = mc.LineCollection(lines, colors='green', linewidths=2)
    lc = Line3DCollection(lines, colors='green', linewidths=2)
    ax.add_collection(lc)

    if path is not None:
        paths = [(path[i], path[i+1]) for i in range(len(path)-1)]
        #lc2 = mc.LineCollection(paths, colors='blue', linewidths=3)
        lc2 = Line3DCollection(paths, colors='blue', linewidths=3)
        ax.add_collection(lc2)

    ax.autoscale()
    ax.margins(0.1)
    plt.show()


def pathSearch(startpos, endpos, obstacles, n_iter, radius, stepSize):
    G = RRT_star(startpos, endpos, obstacles, n_iter, radius, stepSize)
    if G.success:
        path = dijkstra(G)
        # plot(G, obstacles, radius, path)
        return path

In [52]:
# 3D version in progress
if __name__ == '__main__':
    startpos = (0., 0., 0.)
    endpos = (5., 5., 5.)
    obstacles = [(1., 1., 1.), (2., 2., 2.)]
    n_iter = 400
    radius = 0.5
    stepSize = 0.7

    G = RRT_star(startpos, endpos, obstacles, n_iter, radius, stepSize)
    # G = RRT(startpos, endpos, obstacles, n_iter, radius, stepSize)

    if G.success:
        path = dijkstra(G)
        print(path)
        plot(G, obstacles, radius, path)
    else:
        print("No path")
        plot(G, obstacles, radius)
        

[(0.0, 0.0, 0.0), (-0.2211576968233043, 0.4229336229003891, 0.5120707214400789), (0.04321369415730528, 0.9876058708377007, 0.8302735184742991), (0.2866883480969198, 1.5703132827059993, 1.1322208069912416), (0.7187592348355044, 1.4941827808669477, 1.6776737339452246), (0.6217636601884026, 1.3305708435926171, 2.3513375911323378), (0.9915897730588943, 0.9952440990906943, 2.842035686360804), (1.4963156319952007, 0.997450391822246, 3.3270583039709423), (1.923239118214929, 1.5363121629052172, 3.458832082012391), (2.617089722202641, 1.6244711402301786, 3.4871045785930654), (2.655347530698978, 1.8464055019904704, 4.149887709910625), (2.8089125487198534, 1.9285004820500937, 4.827883432234342), (3.2768018713908753, 2.2021374638942994, 5.270830814998532), (3.6520575741084316, 2.762805580840893, 5.457470629125941), (3.502038052261497, 3.3742972414380623, 5.763366186604197), (3.82438497817492, 3.9154855413393848, 5.458066852500629), (4.295388943799416, 4.420840481243159, 5.3450551315118275), (5.0, 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(1, 0), (2, 1), (3, 0), (4, 2), (5, 0), (6, 4), (7, 6), (8, 7), (9, 3), (10, 4), (11, 8), (12, 7), (13, 11), (14, 11), (15, 9), (16, 5), (17, 5), (18, 13), (19, 12), (20, 18), (21, 15), (22, 19), (23, 20), (24, 22), (25, 23), (26, 25), (27, 18), (28, 10), (29, 14), (30, 22), (31, 28), (32, 23), (33, 29), (34, 31), (35, 34), (36, 16), (37, 33), (38, 35), (39, 31), (40, 24), (41, 38), (42, 38), (43, 42), (44, 43), (45, 44), (46, 41), (47, 26), (48, 43), (49, 42), (50, 47), (51, 46), (52, 45), (53, 21), (54, 26), (55, 45), (56, 53), (57, 9), (58, 52), (59, 46), (60, 51), (61, 16), (62, 48), (63, 27), (64, 51), (65, 29), (66, 50), (67, 62), (68, 34), (69, 54), (70, 66), (71, 19), (72, 70), (73, 72), (74, 67), (75, 50), (76, 74), (76, 77), (78, 76), (79, 58), (80, 61), (81, 57), (82, 78), (83, 55), (84, 39), (85, 30), (86, 62), (87, 79), (88, 18), (89, 81), (90, 41), (91, 60), (92, 72), (93, 45), (94, 87), (95, 64), (96, 15), (97, 22), (98, 85), (99, 82), (100, 23), (101, 93), (102, 86), (

In [129]:
#4D version 
def newVertex(randvex, nearvex, stepSize):
    dirn = np.array(randvex) - np.array(nearvex)
    length = np.linalg.norm(dirn)
    dirn = (dirn / length) * min (stepSize, length)

    newvex = (nearvex[0]+dirn[0], nearvex[1]+dirn[1], nearvex[2] + dirn[2], nearvex[3] + dirn[3])
    return newvex
# increased constraints
def window(startpos, endpos):
    ''' Define seach window - 2 times of start to end rectangle'''
    width = endpos[0] - startpos[0]
    height = endpos[1] - startpos[1]
    length = endpos[2] - startpos[2]
    winx = startpos[0] - (width / 2.)
    winy = startpos[1] - (height / 2.)
    winz =  startpos[2] - (length / 2.)
    return winx, winy, winz, width, height, length


def isInWindow(pos, winx, winy, winz, width, height, length):
    ''' Restrict new vertex insides search window'''
    if winx < pos[0] < winx+width and \
        winy < pos[1] < winy+height and \
            winz < pos[2] < winz+length:
        return True
    else:
        return False


class Graph:
    ''' Define graph '''
    def __init__(self, startpos, endpos):
        self.startpos = startpos
        self.endpos = endpos

        self.vertices = [startpos]
        self.edges = []
        self.success = False

        self.vex2idx = {startpos:0}
        self.neighbors = {0:[]}
        self.distances = {0:0.}

        self.sx = endpos[0] - startpos[0]
        self.sy = endpos[1] - startpos[1]
        self.sz = endpos[2] - startpos[2]
        self.sw = endpos[3] - startpos[3] #added w

    def add_vex(self, pos):
        try:
            idx = self.vex2idx[pos]
        except:
            idx = len(self.vertices)
            self.vertices.append(pos)
            self.vex2idx[pos] = idx
            self.neighbors[idx] = []
        return idx

    def add_edge(self, idx1, idx2, cost):
        self.edges.append((idx1, idx2))
        self.neighbors[idx1].append((idx2, cost))
        self.neighbors[idx2].append((idx1, cost))


    def randomPosition(self):
        rx = random()
        ry = random()
        rz = random()
        rw = random()

        posx = self.startpos[0] - (self.sx / 2.) + rx * self.sx * 2
        posy = self.startpos[1] - (self.sy / 2.) + ry * self.sy * 2
        posz = self.startpos[2] - (self.sz / 2.) + rz * self.sz * 2
        posw = self.startpos[3] - (self.sw / 2.) + rw * self.sw * 2
        return posx, posy, posz, posw


def plt_sphere(list_center, list_radius, ax):
    for c, r in zip(list_center, list_radius):
        # draw sphere
        u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
        x = r*np.cos(u)*np.sin(v)
        y = r*np.sin(u)*np.sin(v)
        z = r*np.cos(v)
        ax.plot_surface(c[0] - x, c[1] - y, c[2] - z, color="red")


def plot(G, obstacles, radius, path=None):
    '''
    Plot RRT, obstacles and shortest path
    '''
    plt.close()
    fig = plt.figure(figsize=(8, 10))
    fig.add_subplot(211)
#     px = [x for x, y, z,w in G.vertices]
#     py = [y for x, y, z,w in G.vertices]
#     pz = [z for x, y, z,w in G.vertices]
    
    
    ax = plt.axes(projection='3d')
    
    plt_sphere([obs[:3] for obs in obstacles], [radius]*len(obstacles), ax)
    
#     ax.scatter3D(px, py, pz, c='cyan')
    ax.scatter3D(G.startpos[0], G.startpos[1], G.startpos[2], c='black')
    ax.scatter3D(G.endpos[0], G.endpos[1], G.endpos[2], c='black')
    
#     lines = [(G.vertices[edge[0]][:3], G.vertices[edge[1]][:3]) for edge in G.edges]
#      #lc = mc.LineCollection(lines, colors='green', linewidths=2)
#     lc = Line3DCollection(lines, colors='green', linewidths=2)
#     ax.add_collection(lc)

    if path is not None:
        paths = [(path[i][:3], path[i+1][:3]) for i in range(len(path)-1)]
        #lc2 = mc.LineCollection(paths, colors='blue', linewidths=3)
        lc2 = Line3DCollection(paths, colors='blue', linewidths=3)
        ax.add_collection(lc2)

    #ax.autoscale()
    ax.margins(0.2)
    
    fig.add_subplot(212)
    plt.plot([w for x, y, z,w in path])
    plt.subplots_adjust(hspace = 5)
    plt.xlabel('timestep')
    plt.ylabel('yaw')
    plt.show()

In [130]:
if __name__ == '__main__':
    startpos = (0., 0., 0., 90.) # define things for search
    endpos = (5., 5., 5., 85.)
    obstacles = [(1., 1., 1., 90.), (2., 2., 2.,90.)]
    n_iter = 800
    radius = 1.5
    stepSize = 0.7

    G = RRT_star(startpos, endpos, obstacles, n_iter, radius, stepSize)
    # G = RRT(startpos, endpos, obstacles, n_iter, radius, stepSize)

    if G.success:
        path = dijkstra(G)
        print(path)
        
        plot(G, obstacles, radius, path)
    else:
        print("No path")
        plot(G, obstacles, radius)
        

[(0.0, 0.0, 0.0, 90.0), (-0.22120241586290737, 0.17058518488191718, 0.06257082565133312, 89.36120811081189), (0.36277155950494955, -0.011088825300766048, 0.03711616735602692, 89.02161863734683), (0.8875299130642987, -0.012616826841357065, -0.17401341771895662, 88.60924667674932), (1.4316417018645673, 0.33252979230696067, 0.06647529856429327, 88.73955918543322), (1.9336955279328376, 0.7606260269676954, 0.3000424821908847, 88.75060184508813), (2.384574601829166, 1.2345628164591618, 0.12684450409639889, 88.92975105851897), (2.6565566597040555, 1.6440632407032072, 0.3244194370827927, 88.47225864481848), (2.811097668873653, 2.0100587629854987, 0.7608510250229744, 88.09583890144393), (2.8744499868525635, 2.6229577627675704, 0.894208481918461, 88.40007086650218), (2.9080438999394542, 3.0377491639639373, 1.3618411575369793, 88.08679906248699), (3.380250930716003, 3.1183072642771066, 1.8575382033141907, 88.20851739046957), (3.575250827166281, 2.9675548009448782, 2.501954467327657, 88.3267392744

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
from itertools import starmap
def length_4D(path):
        pure_p = [p[:3] for p in path]
        pairs=zip(pure_p, pure_p[1:])
        return sum(starmap(distance,pairs))

def distance_4D(p1, p2):
    # calculates distance between two given points p1 and p2
    return math.sqrt((p1.sx-p2.sx)**2 + (p1.sy-p2.sy)**2 + (p1.sz-p2.sz)**2)

In [122]:
length_4D(path)

12.487526966757608

In [ ]:
def path_to_waypoint(path):
    wp = []
    for node in path:
        new_waypoint = {'frame': MavrosDrone.FRAME_REFERENCE.RELATIVE_ALT.value,
                            'command': MavrosDrone.MAV_CMD.NAVIGATE_TO_WAYPOINT.value,
                            'is_current': False, 'autocontinue': True, 'param1': 0,
                            'param2': 0, 'param3': 0, 'x_lat': node[0],
                            'y_long': node[1],
                            'z_alt': node[2]}
        wp.append(new_waypoint)
    return wp